<a href="https://colab.research.google.com/github/shillpak/MachineLearning/blob/main/Chatbot_inprogress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gradio


In [9]:
!pip install PyMuPDF
!pip install transformers
!pip install sentence_transformers
!pip install faiss-cpu
!pip install nltk

In [11]:
import gradio as gr
import fitz
from transformers import BartTokenizer, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer
import faiss
import nltk

In [ ]:
nltk.download("punkt")
from nltk.tokenize import sent_tokenize

In [ ]:
# --- Load model and tokenizer ---
embedder = SentenceTransformer('all-MiniLM-L6-v2')
generator = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

In [ ]:
# --- Load and process PDF ---
def extract_and_index(pdf_file):
    doc = fitz.open(pdf_file.name)
    text = ""
    for page in doc:
        text += page.get_text()

    sentences = sent_tokenize(text)
    chunks = [" ".join(sentences[i:i + 5]) for i in range(0, len(sentences), 5)]

    embeddings = embedder.encode(chunks, convert_to_numpy=True).astype("float32")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return chunks, index

chunks, index = [], None

def answer_question(query):
    query_vec = embedder.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(query_vec, k=3)
    context = " ".join([chunks[i] for i in indices[0]])

    input_text = query + " " + context
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=1024)
    output = generator.generate(**inputs, max_length=100, num_beams=5)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def chatbot_interface(pdf_file, query):
    global chunks, index
    if not index:
        chunks, index = extract_and_index(pdf_file)
    return answer_question(query)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.3/31.3 MB 10.8 MB/s eta 0:00:00


In [ ]:
import gradio as gr
import fitz
from transformers import BartTokenizer, BartForConditionalGeneration
from sentence_transformers import SentenceTransformer
import faiss
import nltk

In [12]:
# --- Load model and tokenizer ---
embedder = SentenceTransformer('all-MiniLM-L6-v2')
generator = BartForConditionalGeneration.from_pretrained('facebook/bart-large')
tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [13]:
# --- Load and process PDF ---
def extract_and_index(pdf_file):
    doc = fitz.open(pdf_file.name)
    text = ""
    for page in doc:
        text += page.get_text()

    sentences = sent_tokenize(text)
    chunks = [" ".join(sentences[i:i + 5]) for i in range(0, len(sentences), 5)]

    embeddings = embedder.encode(chunks, convert_to_numpy=True).astype("float32")
    index = faiss.IndexFlatL2(embeddings.shape[1])
    index.add(embeddings)
    return chunks, index

chunks, index = [], None

def answer_question(query):
    query_vec = embedder.encode([query], convert_to_numpy=True).astype("float32")
    distances, indices = index.search(query_vec, k=3)
    context = " ".join([chunks[i] for i in indices[0]])

    input_text = query + " " + context
    inputs = tokenizer(input_text, return_tensors="pt", truncation=True, max_length=1024)
    output = generator.generate(**inputs, max_length=100, num_beams=5)
    return tokenizer.decode(output[0], skip_special_tokens=True)

def chatbot_interface(pdf_file, query):
    global chunks, index
    if not index:
        chunks, index = extract_and_index(pdf_file)
    return answer_question(query)